In [1]:
import numpy as np
import cv2
import task1
import matplotlib.pyplot as plt
import copy

In [2]:
img = cv2.imread("./data/proj1-task2.jpg")#,cv2.IMREAD_GRAYSCALE)

In [3]:
def show_image(img, delay=10000):
    cv2.imshow('IMG',img)
    cv2.waitKey(delay)
    cv2.destroyAllWindows()

In [4]:
a_template = img[136:146, 240:248]
c_template = img[215:223, 122:129]
b_template = img[187:201,614:622]

In [5]:
def templateMatching(img, template):
    img = img.copy()
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    template = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)
    w, h = template.T.shape
    w, h = w, h
    res = cv2.matchTemplate(img_gray,template,cv2.TM_CCOEFF_NORMED)
    threshold = 0.7
    loc = np.where( res >= threshold)
    cnt = 0
    for pt in zip(*loc[::-1]):
        cv2.rectangle(img, pt, (pt[0] + w, pt[1] + h), (0,0,255), 2)
        cnt+=1
    print(cnt)
    
    return img

# cv2.imwrite('res.png',img_rgb)

In [6]:
# show_image(img,0)

# show_image(templateMatching(img, a_template))

# show_image(templateMatching(img, b_template))

# show_image(templateMatching(img, c_template),0)

# template = cv2.cvtColor(a_template, cv2.COLOR_BGR2GRAY)

In [7]:
#Reference - https://docs.opencv.org/2.4/doc/tutorials/imgproc/histograms/template_matching/template_matching.html
def get_ccoeff_normed(img, template):
    
    arr_element_sub = lambda arr, val: [arr_val - val for arr_val in arr]
    element_squared_sum = lambda arr: sum(z**2 for z in arr)
    sum_normed = lambda arr, const: sum(arr) / const
    sum_of_prods = lambda arr1, arr2: sum([a1_val * a2_val for a1_val, a2_val in zip(arr1, arr2)])
    flatten = lambda arr : [arr_val for arr_row in arr for arr_val in arr_row]

    w ,h =  len(template), len(template[0]),
    wh_prod = w*h*1.0
    
    ccoeff_normed = [[0.0 for _ in range(len(img[0]) - h + 1)] for _ in range(len(img) - w + 1)]
#     print("w %d h %d ccnormed_len %d ccnormed_len %d" %(w,h, len(ccoeff_normed),len(ccoeff_normed[0])))

    #Is same for every sliding window as template doesn't change
    template = flatten(template)
    t_sum_normed = sum_normed(template, wh_prod)
    t_dash = arr_element_sub(template, t_sum_normed)
    t_dash_sum_squared = element_squared_sum(t_dash)

    #Hack for zeros
    if(t_dash_sum_squared == 0):
        t_dash_sum_squared = 1e-6
#     print('t_sum_normed %f t_dash %s t_dash_sum_sq %d'%(t_sum_normed, t_dash, t_dash_sum_squared))

    def get_ccoeff_normed_val(i, j,):
#         nonlocal img, template
#         nonlocal t_sum_normed

        #Flatten the image
        im = [img[x][y] for x in range(i, i + w) for y in range(j, j + h)]
#         print('Im',im)

        #changes with the window
        #check for indices
        i_sum_normed = sum_normed(im, wh_prod)
        i_dash = arr_element_sub(im, i_sum_normed)
        i_dash_sum_squared = element_squared_sum(i_dash)
        if i_dash_sum_squared == 0:
            i_dash_sum_squared = 1e-6
    
        
#         print('i_sum_normed %d \nimg = %s\n i_dash %s\n i_dash_sum_sq %d'%(i_sum_normed, im, i_dash, t_dash_sum_squared))        
#         print('t_Val=%d i_val=%d'%(len(t_dash), len(i_dash)))
#         input()
        
        numerator = sum_of_prods(t_dash, i_dash)
        denominator = np.sqrt(t_dash_sum_squared * i_dash_sum_squared)
        
#         print(numerator, denominator)
        return numerator/denominator
    
    for x in range(len(ccoeff_normed) - w):
        for y in range(len(ccoeff_normed[0]) - h):
            ccoeff_normed[x][y] = get_ccoeff_normed_val(x, y)
    return ccoeff_normed

In [8]:
templateList = cv2.cvtColor(a_template, cv2.COLOR_BGR2GRAY).tolist()
imgList = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY).tolist()

In [9]:
# # temp_c = cv2.cvtColor(c_template, cv2.COLOR_BGR2GRAY)

# # mul = lambda x : i*
# t_sum_normed = temp_c.sum()/ np.prod(temp_c.shape)
# t_dash = temp_c.flatten() - t_sum_normed
# t_dash_sum_squared = np.square(t_dash).sum()
# print('t_sum_normed',t_sum_normed)
# print('t_dash \n',t_dash)
# print('t_dash_sum_squared',t_dash_sum_squared)

In [14]:
ccoeff_normed = get_ccoeff_normed(imgList, templateList)
threshold = 0.7
coordinates = [(y,x) for x in range(len(ccoeff_normed)) for y in range(len(ccoeff_normed[0])) if ccoeff_normed[x][y] >= threshold]

In [11]:
z = cv2.matchTemplate(np.array(imgList, dtype=np.float32), np.array(templateList, dtype=np.float32), cv2.TM_CCOEFF_NORMED)

In [24]:
coordinates_z = [(y,x) for x in range(len(z)) for y in range(len(z[0])) if z[x][y] >= threshold]

In [23]:
coordinates

[(281, 55),
 (341, 55),
 (281, 56),
 (341, 56),
 (293, 88),
 (240, 136),
 (414, 136),
 (415, 136),
 (456, 136),
 (520, 136),
 (569, 136),
 (570, 136),
 (514, 161),
 (430, 189),
 (183, 214),
 (184, 214),
 (264, 214),
 (265, 214),
 (585, 214),
 (253, 238),
 (289, 238),
 (107, 269)]

In [25]:
coordinates_z

[(281, 55),
 (341, 55),
 (281, 56),
 (341, 56),
 (293, 88),
 (240, 136),
 (414, 136),
 (415, 136),
 (456, 136),
 (520, 136),
 (569, 136),
 (570, 136),
 (514, 161),
 (430, 189),
 (183, 214),
 (184, 214),
 (264, 214),
 (265, 214),
 (585, 214),
 (253, 238),
 (289, 238),
 (107, 269)]